In [ ]:
#pip install pyarrow
#pip install fastparquet

#### this notebook is responsible for the preprocessing of weather-related data.

In [1]:
import os
directory = os.getcwd()  
directory = directory.split('src')
path = directory[0]
%cd $path
%pwd

c:\Users\JENMOSQU\Documents\PERSONAL\cod\Probabilistic-Electricity-Demand-


'c:\\Users\\JENMOSQU\\Documents\\PERSONAL\\cod\\Probabilistic-Electricity-Demand-'

In [2]:
import os
import pandas as pd
import plotly.express as px


#local module
from src.Utilities import utilities as tool


# Configuración options.display
# ==============================================================================
pd.set_option('display.max_rows', 5000)
pd.set_option("display.float_format", "{:.2f}".format)

pd.options.display.max_rows = 5000
pd.options.display.max_columns = 5000
pd.options.display.max_colwidth = 5000
pd.options.display.max_dir_items = 5000
pd.options.display.width = 5000

Using TensorFlow backend


### Filtra los archivos .data en la ruta especificada por fecha y exporta el dataframe resultante en formato parquet.

In [4]:
# datos de temperatura
#ruta_archivos="C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/temperatura"
ruta_archivos="C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/nuevo_20231028/TemperaturaNacionalDiaria"
tool.demand_utilities.filtrar_exportar_parquet(ruta_archivos, "2000-01-01", "2023-10-01", "temperatura", "TSSM_CON@") #"2023-03-01"
print("Archivo parquet es  guardado en:", os.path.join(ruta_archivos, "temperatura.parquet"))

#Datos de precipitación
#ruta_archivos_prep="C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/precipitación"
ruta_archivos_prep='C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/nuevo_20231028/PrecipitacionNacionalDiaria'
tool.demand_utilities.filtrar_exportar_parquet(ruta_archivos_prep, "2000-01-01", "2023-10-01", "precip","PTPM_CON_INTER@") #Pronóstico Térmico de Punto Medio con Interconexión: "Pronóstico Térmico de Punto Medio con Interconexión", y se refiere a un modelo numérico que combina la información de la temperatura y la humedad atmosférica para generar pronósticos de precipitación en una región determinada.
print("Archivo parquet guardado en:", os.path.join(ruta_archivos_prep, "precip.parquet")) #se mide comúnmente en milímetros (mm) o en litros por metro cuadrado (l/m²)

Archivo parquet es  guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/nuevo_20231028/TemperaturaNacionalDiaria\temperatura.parquet
Archivo parquet guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/nuevo_20231028/PrecipitacionNacionalDiaria\precip.parquet


In [2]:
import pandas as pd

In [45]:
temperatura = pd.read_parquet('C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/nuevo_20231028/TemperaturaNacionalDiaria/temperatura.parquet')
temperatura.head(2)
temperatura.tail(2)
temperatura[temperatura['Fecha']>'2023-06-5 07:00:00']

,Fecha,Valor
36249,2023-06-05 13:00:00,24.0
36250,2023-06-05 18:00:00,19.4
36251,2023-06-06 07:00:00,16.6
40754,2023-06-05 13:00:00,32.8
40755,2023-06-05 19:00:00,29.0
...,...,...
39060,2023-06-05 19:00:00,10.6
39061,2023-06-06 07:00:00,10.2
37230,2023-06-05 13:00:00,24.4
37231,2023-06-05 18:00:00,22.0


In [8]:
prep=pd.read_parquet('C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/nuevo_20231028/PrecipitacionNacionalDiaria/precip.parquet')
prep.tail(5)

,Fecha,Valor
11417,2021-12-21 07:00:00,5.0
11418,2021-12-23 07:00:00,12.0
11419,2021-12-24 07:00:00,35.0
11420,2021-12-25 07:00:00,2.0
11421,2021-12-26 07:00:00,8.0


In [42]:
filtrado=prep[prep['Fecha']>'2023-06-5 07:00:00']
filtrado.head()

,Fecha,Valor
21369,2023-06-06 07:00:00,0.3
14290,2023-06-06 07:00:00,0.1
21453,2023-06-06 07:00:00,0.5
17848,2023-06-06 07:00:00,0.3
29513,2023-06-06 07:00:00,0.3


### Agrupa los datos por el intervalo de tiempo especificado y calcula la media de los datos en ese intervalo.

In [25]:
filepath_prep = "C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/precipitación/precip.parquet"
filepath_temp = "C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/temperatura/temperatura.parquet"
column = "Fecha"
resampling = "D" #diario/ "D" para días, "H" para horas

df_resampled_prep = tool.demand_utilities.read_parquet_file_and_resample(filepath_prep, column, resampling)
df_resampled_temp = tool.demand_utilities.read_parquet_file_and_resample(filepath_temp, column, resampling)

df_resampled_prep.reset_index(inplace=True)
df_resampled_temp.reset_index(inplace=True)

In [7]:
df_resampled_temp

,Fecha,Valor
0,2000-01-01,22.26
1,2000-01-02,22.10
2,2000-01-03,21.87
3,2000-01-04,21.58
4,2000-01-05,21.51
...,...,...
8455,2023-02-24,22.30
8456,2023-02-25,22.65
8457,2023-02-26,22.56
8458,2023-02-27,22.70


In [8]:
df_resampled_prep

,Fecha,Valor
0,2000-01-01,2.65
1,2000-01-02,2.77
2,2000-01-03,3.76
3,2000-01-04,4.03
4,2000-01-05,3.46
...,...,...
8455,2023-02-24,1.16
8456,2023-02-25,0.58
8457,2023-02-26,1.52
8458,2023-02-27,2.32


In [9]:
ruta_archivos = "C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894"

elementos=['HRA2_MN_D','HRA2_MX_D','PT_10_MX_D','PT_10_TT_D','PT_2_MX_D','PT_2_TT_D','PTPG_TT_D','TA10_MX_10_D','TAMX2_MN_D','TA10_MEDIA_D','TA10_MN_D','TA10_MX_D','TAMN10_MEDIA_D','TAMN10_MN_D','TAMN10_MX_D','TAMX10_MEDIA_D','TAMX10_MN_D','TAMX10_MX_10_D','TAMX10_MX_D','TAMX2_MEDIA_D','TAMX2_MX_10_D','TAMX2_MX_D','THSM_MEDIA_D','THSM_MN_D','THSM_MX_D',
           'TSSM_MEDIA_D','TSSM_MN_D','TSSM_MX_D','TSTG_MEDIA_D','TSTG_MN_D','TSTG_MX_D','VV_10_MEDIA_D','VV_10_VECT_MEDIA_D','VVAG_MEDIA_D','VVMX_MX_D']

dfs = {}  # Un diccionario para almacenar los DataFrames
filepaths = {}  # Un diccionario para almacenar las rutas de los archivos

for elemento in elementos:
    tool.demand_utilities.filtrar_exportar_parquet(ruta_archivos, "2017-12-01", "2023-07-31", elemento, f"{elemento}@")
    print(f"Archivo parquet de {elemento} es guardado en:", os.path.join(ruta_archivos, f"{elemento}.parquet"))

    filepath_elemento = os.path.join(ruta_archivos, f"{elemento}.parquet")
    column = "Fecha"
    resampling = "D"  # diario/ "D" para días, "H" para horas, "30T" para intervalos de 30 minutos

    df_resampled_elemento = tool.demand_utilities.read_parquet_file_and_resample(filepath_elemento, column, resampling)
    df_resampled_elemento.reset_index(inplace=True)

    # Convierte las cadenas en objetos Timestamp
    fecha_inicio = pd.Timestamp("2017-12-01")
    fecha_fin = pd.Timestamp("2023-07-31")

    # Verifica si el DataFrame no tiene valores NaN en el rango de fechas deseado
    if df_resampled_elemento["Fecha"].min() <= fecha_inicio and df_resampled_elemento["Fecha"].max() >= fecha_fin and not df_resampled_elemento.isna().values.any():
        # Almacena el DataFrame en el diccionario dfs y la ruta en el diccionario filepaths
        dfs[elemento] = df_resampled_elemento
        filepaths[elemento] = filepath_elemento

Archivo parquet de HRA2_MN_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894\HRA2_MN_D.parquet
Archivo parquet de HRA2_MX_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894\HRA2_MX_D.parquet
Archivo parquet de PT_10_MX_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894\PT_10_MX_D.parquet
Archivo parquet de PT_10_TT_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894\PT_10_TT_D.parquet
Archivo parquet de PT_2_MX_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894\PT_2_MX_D.parquet
Archivo parquet de PT_2_TT_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894\PT_2_TT_D.parquet
Archivo parquet de PTPG_TT_D es guardado en: C:/Users/JENMOS

In [10]:
# Concatenar cada elemento de dfs en un solo DataFrame
df_weather = pd.concat(dfs.values(), axis=1, keys=dfs.keys())
# Asegurarse de que las fechas sean únicas en el DataFrame resultante
df_weather = df_weather.loc[:, ~df_weather.columns.duplicated()]
# Reorganizar las columnas
df_weather.columns = df_weather.columns.map('_'.join)
# Restablecer el nombre de la columna de "Fecha"
df_weather = df_weather.rename(columns={'Fecha_Fecha': 'Fecha'})
# Supongamos que deseas eliminar las columnas "columna1" y "columna2"
df_weather.drop(columns=['VV_10_MEDIA_D_Fecha', 'VV_10_VECT_MEDIA_D_Fecha', 'VVAG_MEDIA_D_Fecha'], inplace=True)
df_weather = df_weather.rename(columns={'PT_10_TT_D_Fecha': 'FECHA','PT_10_TT_D_Valor':'PT_10_TT_D','VV_10_MEDIA_D_Valor':'VV_10_MEDIA_D','VV_10_VECT_MEDIA_D_Valor':'VV_10_VECT_MEDIA_D','VVAG_MEDIA_D_Valor':'VVAG_MEDIA_D'})
df_weather.head()

,FECHA,PT_10_TT_D,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,VVAG_MEDIA_D
0,2017-12-01,6.23,1.70,0.93,1.75
1,2017-12-02,7.89,1.76,0.93,1.77
2,2017-12-03,5.62,1.85,1.15,2.08
3,2017-12-04,4.21,2.03,1.34,2.10
4,2017-12-05,4.82,2.28,1.55,2.21


In [26]:
df_resampled_prep=df_resampled_prep[(df_resampled_prep['Fecha'] >= '2017-12-01') & (df_resampled_prep['Fecha'] <= '2023-07-31')]
df_resampled_prep.rename(columns={'Fecha': 'FECHA'}, inplace=True)

PT_10_TT_D=df_weather[['FECHA','PT_10_TT_D']]

In [31]:
df_resampled_prep=df_resampled_prep[(df_resampled_prep['Fecha'] >= '2017-12-01') & (df_resampled_prep['Fecha'] <= '2023-07-31')]
df_resampled_prep.rename(columns={'Fecha': 'FECHA'}, inplace=True)

PT_10_TT_D=df_weather[['FECHA','PT_10_TT_D']]


merge=pd.merge(PT_10_TT_D, df_resampled_prep, on='FECHA', how='left')
merge.head()

,FECHA,PT_10_TT_D,Valor
0,2017-12-01,6.23,9.59
1,2017-12-02,7.89,7.11
2,2017-12-03,5.62,3.98
3,2017-12-04,4.21,4.09
4,2017-12-05,4.82,2.39


In [39]:
fig = px.line(merge, x='FECHA', y='PT_10_TT_D', markers=True, line_shape='linear', title='Precipitación total nueva')
fig.update_xaxes(title='Fecha')
fig.update_yaxes(title='precipitación')
fig.add_trace(px.line(merge, x='FECHA', y='Valor').update_traces(line=dict(color='red')).data[0])

fig.show()

In [40]:
velocidad_viento = df_weather.drop(columns='PT_10_TT_D')
velocidad_viento.head()

#Escogida: VV_10_VECT_MEDIA_D: Persistencia diaria
#Escogida: VVAG_MEDIA_D: Velocidad del viento media diaria

,FECHA,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,VVAG_MEDIA_D
0,2017-12-01,1.70,0.93,1.75
1,2017-12-02,1.76,0.93,1.77
2,2017-12-03,1.85,1.15,2.08
3,2017-12-04,2.03,1.34,2.10
4,2017-12-05,2.28,1.55,2.21


In [44]:
fig = px.line(velocidad_viento, x='FECHA', y='VV_10_MEDIA_D', markers=True, line_shape='linear', title='Precipitación total nueva')
fig.update_xaxes(title='Fecha')
fig.update_yaxes(title='precipitación')
fig.add_trace(px.line(velocidad_viento, x='FECHA', y='VV_10_VECT_MEDIA_D').update_traces(line=dict(color='#805c77')).data[0])
fig.add_trace(px.line(velocidad_viento, x='FECHA', y='VVAG_MEDIA_D').update_traces(line=dict(color='#75b872')).data[0])

fig.show()

### Convierte un dataframe de datos diarios a datos mensuales

In [20]:
datos_mensuales_prep = tool.demand_utilities.datos_diarios_a_mensuales(df_resampled_prep, columna_demanda='Valor',agregation_operation='mean')
datos_mensuales_temp = tool.demand_utilities.datos_diarios_a_mensuales(df_resampled_temp, columna_demanda='Valor',agregation_operation='mean')

### Comparación de dataframes

In [21]:
tool.demand_utilities.comparar_dataframes(datos_mensuales_temp,datos_mensuales_prep)

Las siguientes celdas son diferentes:
 - Fila 0, columna 2: 21.70683103612097 != 3.758937055079399
 - Fila 1, columna 2: 21.87987963531676 != 4.935296989591993
 - Fila 2, columna 2: 22.014621083769317 != 4.732167989632687
 - Fila 3, columna 2: 22.13855749868336 != 6.154437729242565
 - Fila 4, columna 2: 22.065471863076443 != 8.85318212516916
 - Fila 5, columna 2: 22.15659772003878 != 6.763235887657237
 - Fila 6, columna 2: 22.08342777041746 != 5.877186701772555
 - Fila 7, columna 2: 22.35675155644346 != 5.728207506278101
 - Fila 8, columna 2: 21.77471429904975 != 8.12273536409564
 - Fila 9, columna 2: 22.212557191961412 != 6.707036873567598
 - Fila 10, columna 2: 22.108868901209362 != 5.443199461265679
 - Fila 11, columna 2: 21.909017370853025 != 4.054063174985304
 - Fila 12, columna 2: 21.97306205664559 != 2.221235278625576
 - Fila 13, columna 2: 22.4005381239591 != 2.308588787088499
 - Fila 14, columna 2: 22.21855894374678 != 4.648875486785825
 - Fila 15, columna 2: 22.57812897888840

False

### Exportar dataframes resultantes en formato parquet 

In [22]:
# Exportar dataframe resultante en formato parquet: datos_mensuales_temp

filepath_temp_mensuales = "C:/Users/JENMOSQU/Documents/PERSONAL/Datos-tesis/CLIMA/temperatura/"
nombre_archivo="datos_mensuales_temp"
ruta_archivo_parquet = os.path.join(filepath_temp_mensuales, f"{nombre_archivo}.parquet")
datos_mensuales_temp.to_parquet(ruta_archivo_parquet)
print("Archivo parquet guardado en:", os.path.join(filepath_temp_mensuales, nombre_archivo))


# Exportar dataframe resultante en formato parquet: datos_mensuales_prep

filepath_prep_mensuales = "C:/Users/JENMOSQU/Documents/PERSONAL/Datos-tesis/CLIMA/precipitación"
nombre_archivo="datos_mensuales_prep"
ruta_archivo_parquet = os.path.join(filepath_prep_mensuales, f"{nombre_archivo}.parquet")
datos_mensuales_prep.to_parquet(ruta_archivo_parquet)
print("Archivo parquet guardado en:", os.path.join(filepath_prep_mensuales, nombre_archivo))

Archivo parquet guardado en: C:/Users/JENMOSQU/Documents/PERSONAL/Datos-tesis/CLIMA/temperatura/datos_mensuales_temp
Archivo parquet guardado en: C:/Users/JENMOSQU/Documents/PERSONAL/Datos-tesis/CLIMA/precipitación\datos_mensuales_prep


### Verificación de agregación de datos

In [24]:
# Contar el número de observaciones por día
# df_count = df_resampled_temp.resample("D").count()
# print(df_count)

# # Agrupar por año y contar el número de filas
# df_resampled_temp.reset_index(inplace=True)
# conteo_por_anio = df_resampled_temp.groupby(df_resampled_temp['Fecha'].dt.year)['Valor'].count()

# # Mostrar el resultado
# print(conteo_por_anio)

In [140]:
# Agregamos una columna con la hora de la fecha
# df['Hora'] = df['Fecha'].dt.hour
# df.head()
# Ordenamos por la columna 'Hora'
#df = df.sort_values('Hora')


In [ ]:
# # VERIFICACIÓN 

# # Filtramos por el día
# dia = '2000-01-01'
# df_filtrado = df[df['Fecha'].dt.date == pd.to_datetime(dia).date()]

# # Imprimimos el dataframe filtrado y ordenado por hora
# print(df_filtrado)

# #filtrar por año
# df_final.loc[df_final['Fecha'].dt.year == 2000]
# # Mostrar el resultado
# print(df_filtrado)


# num_filas_por_fecha = df['Fecha'].value_counts()
# num_filas_por_fecha